In [2]:
from abc import ABC,abstractmethod

class NotificationSender(ABC):
    @abstractmethod
    def send(self,messege) ->str: ...

class EmailNotification(NotificationSender):
    def send(self,messege):
        return f"[email] sending: {messege}"
    
class SMSNotification(NotificationSender):
    def send(self,messege):
        return f"[sms] sending: {messege}"
    
class Notification(ABC):
    def __init__(self,sender) -> None:
        self.sender = sender

    def notify(self,messege:str): ...
        
class AlertNotification(Notification):
    def notify(self,messege:str):
        return self.sender.send(messege)

email = EmailNotification()
sms = SMSNotification()

alert = AlertNotification(email)
print(alert.notify("система перегружена"))

alert.sender = sms
print(alert.notify("низкий заряд батареи"))

[email] sending: система перегружена
[sms] sending: низкий заряд батареи


In [3]:
class PushNotification:
    def push(self,content:str):
        return f"[push] sending: {content}"

class PushNotificationAdapter(NotificationSender):
    def __init__(self,adaptee:PushNotification) -> None:
        self.adaptee = adaptee

    def send(self,messege) ->str:
        return self.adaptee.push(messege)
    

push = PushNotification()
adapter = PushNotificationAdapter(push)

alert = AlertNotification(adapter)
print(alert.notify("новый способ сообщения"))


[push] sending: новый способ сообщения


In [7]:
class BaseNotification(ABC):
    @abstractmethod
    def notify(self,messege) ->str: ...

class SimpleNotification(BaseNotification):
    def __init__(self,sender) ->None:
        self.sender = sender

    def notify(self,messege) -> str:
        return self.sender.send(messege)
    
class UrgentNotification(BaseNotification):
    def __init__(self,wrapped: BaseNotification) ->None:
        self.wrapped = wrapped

    def notify(self,messege) -> str:
        return "[urgent]"+""+ self.wrapped.notify(messege)
    
email = EmailNotification()
simple = SimpleNotification(email)

urgent = UrgentNotification(simple)
urgent.notify("сервер упал")


'[urgent][email] sending: сервер упал'

In [2]:
class WaterHeater:
    def heat(self):
        print("нагрев воды")

class CoffeeGrinder:
    def grind(self):
        print("молем кофе")

class CoffeeBreaker:
    def brew(self):
        print("варим кофе")

class CoffeeMachine:
    def __init__(self) -> None:
        self.header = WaterHeater()
        self.grider = CoffeeGrinder()
        self.brewer = CoffeeBreaker()

    def make_coffee(self):
        print("-начинаем готовит кофе: ")
        self.header.heat()
        self.grider.grind()
        self.brewer.brew()
        print("кофе готов")


machine = CoffeeMachine()
machine.make_coffee()

-начинаем готовит кофе: 
нагрев воды
молем кофе
варим кофе
кофе готов


In [7]:
class TreeType:
    def __init__(self,name,color,texture) ->None:
        self.name = name
        self.color = color
        self.texture = texture

    def draw(self,x,y):
        print(f"рисуем {self.name} {self.color} в точке x:{x}, y:{y}")

class TreeFactory:
    _tree_types = {}

    @classmethod
    def get_tree_type(cls,name,color,texture):
        key = (name,color,texture)
        if key not in cls._tree_types:
            cls._tree_types[key] = TreeType(name,color,texture)
        return cls._tree_types[key]

class Tree:
    def __init__(self,x,y,tree_type:TreeType) -> None:
        self.x = x
        self.y = y
        self.type = tree_type

    def draw(self):
        self.type.draw(self.x,self.y)

pine_type = TreeFactory.get_tree_type("сосна","зеленый","хвоя.png")

tree1 = Tree(10,20,pine_type)
tree2 = Tree(30,40,pine_type)

tree1.draw()
tree2.draw()

рисуем сосна зеленый в точке x:10, y:20
рисуем сосна зеленый в точке x:30, y:40


In [9]:
class RealDatabase:
    def connect(self):
        print("подключение к базе данных")

    def query(self,sql):
        print(f"выполнение запроса: {sql}")

class DatabaseProxy:
    def __init__(self):
        self.real_bd = None
        self.connected= False

    def connect(self):
        if not self.connected:
            self.real_bd = RealDatabase()
            self.real_bd.connect()
            self.connected = True
        else:
            print("уже подключено")

    def query(self,sql):
        self.connect()
        self.real_bd.query(sql)

db = DatabaseProxy()
db.query("SELECT * FROM users")
db.query("SELECT * FROM products")


подключение к базе данных
выполнение запроса: SELECT * FROM users
уже подключено
выполнение запроса: SELECT * FROM products


In [13]:
from abc import ABC,abstractmethod

class Handler(ABC):
    def __init__(self) -> None:
        self.next_handler = None

    def set_next(self,handler):
        self.next_handler = handler
        return handler
    
    @abstractmethod
    def handle(self,request): ...

class Level1Support(Handler):
    def handle(self, request):
        if request == "пароль":
            print("level 1: помогаем восстановить пароль")

        elif self.next_handler:
            self.next_handler.handle(request)

class Level2Support(Handler):
    def handle(self, request):
        if request == "ошибка программы":
            print("level 2: решает ошибку")

        elif self.next_handler:
            self.next_handler.handle(request)

class Level3Support(Handler):
    def handle(self, request):
        print(f"level 3: эскалация запроса: {request}")

level1 = Level1Support()
level2 = Level2Support()
level3 = Level3Support()

level1.set_next(level2).set_next(level3)

level1.handle("пароль")
level1.handle("ошибка программы")
level1.handle("запрос денег")

level 1: помогаем восстановить пароль
level 2: решает ошибку
level 3: эскалация запроса: запрос денег


In [17]:
from abc import ABC,abstractmethod

class Light:
    def turn_on(self):
        print("свет включен")

    def turn_off(self):
        print("свет выключен")

class Command(ABC):
    @abstractmethod
    def execute(self): ...

class LightOnCommand(Command):
    def __init__(self,light:Light) -> None:
        self.light = light

    def execute(self):
        self.light.turn_on()

class LightOffCommand(Command):
    def __init__(self,light:Light) -> None:
        self.light = light

    def execute(self):
        self.light.turn_off()

class RemoteControl:
    def __init__(self) -> None:
        self.command = None

    def set_command(self,command: Command):
        self.command = command

    def press_button(self):
        if self.command:
            self.command.execute()

light = Light()

on = LightOnCommand(light)
off = LightOffCommand(light)

remote = RemoteControl()
remote.set_command(on)
remote.press_button()

remote.set_command(off)
remote.press_button()

свет включен
свет выключен


In [ ]:
class User:
    def __init__(self,user_id,user_type) -> None:
        self.user_id = user_id
        self.user_type = user_type

    def has_permission(self,permission):
        return self.user_type in permission
    
class BookFlyweight:
    def __init__(self,title,author) -> None:
        self.title = title
        self.author = author
        self.issued = False

    def is_issued(self):
        return self.issued
    
    def issue(self):
        self.issued = True

    def return_book(self):
        self.issued = False

class BookProxy:
    def __init__(self,books):
        self.books = books

    def add_book(self,book_id,title,author):
        if book_id not in self.books:
            self.books[book_id] = BookFlyweight(title,author)

    def check_availability(self,book_id):
        return book_id in self.books and not self.books[book_id].is_issued()

    def issue_book(self,book_id,user_id):
        if self.check_availability(book_id):
            print(f"Book {book_id} выдана пользователю {user_id}")

        else:
            print(f"book {book_id} не доступна")
    
    def return_book(self,book_id,user_id):
        print(f"book {book_id} была возвращена пользователем {user_id}")


class LibraryFacade:
    def __init__(self) -> None:
        self.users = {}
        self.books = {}
        self.book_proxy = BookProxy(self.books)

    def add_book(self,book_id,title,author):
        self.book_proxy.add_book(book_id,title,author)

    def issue_book(self,book_id,user_id):
        self.book_proxy.issue_book(book_id,user_id)

    def return_book(self,book_id,user_id):
        self.book_proxy.return_book(book_id,user_id)

    def check_book_availability(self,book_id):
        return self.book_proxy.check_availability(book_id)

library = LibraryFacade()

library.add_book(1,"title 1","author 1")
library.add_book(2,"title 2","author 2")

user1 = User(1,"reader")
user2 = User(2,"librian")

print(library.check_book_availability(1))

library.issue_book(1,user1.user_id)
library.return_book(1,user1.user_id)

True
Book 1 выдана пользователю 1
book 1 была возвращена пользователем 1


In [3]:
class Myiterator:
    def __init__(self,items) -> None:
        self.items = items
        self.index = 0

    def __iter__(self):
        return self 
    
    def __next__(self):
        if self.index < len(self.items):
            item = self.items[self.index]
            self.index += 1
            return item
        else:
            print("итерация кончилась!")
            raise StopIteration


class MyCollection:
    def __init__(self,items) -> None:
        self.items = items

    def __iter__(self):
        return Myiterator(self.items)
    
collection = MyCollection([1,2,3,4,5])

for item in collection:
    print(item)


1
2
3
4
5
итерация кончилась!


In [5]:
class User:
    def __init__(self,name) -> None:
        self.name = name
        self.mediator = None

    def set_mediator(self,mediator):
        self.mediator = mediator

    def send_messege(self,messege):
        print(f"{self.name} отправил сообщение: {messege}")
        self.mediator.send_messege(self,messege)

    def receive_messege(self,messege):
        print(f"{self.name} получил сообщение: {messege}")

class ChatMediator:
    def __init__(self) -> None:
        self.users = []

    def add_user(self,user):
        self.users.append(user)
        user.set_mediator(self)

    def send_messege(self,sender,messege):
        for user in self.users:
            if user != sender:
                user.receive_messege(messege)


chat_mediator = ChatMediator()

alice = User("Alice")
bob = User("Bob")
jhon = User("Jhon")

chat_mediator.add_user(alice)
chat_mediator.add_user(bob)
chat_mediator.add_user(jhon)

alice.send_messege("hello")

Alice отправил сообщение: hello
Bob получил сообщение: hello
Jhon получил сообщение: hello


In [7]:
class Memento:
    def __init__(self,text) -> None:
        self.__text = text
    
    def get_text(self):
        return self.__text
    
class TextEditor:
    def __init__(self,text="") -> None:
        self.__text = text

    def set_text(self,text):
        self.__text = text

    def get_text(self):
        return self.__text
    
    def creat_memento(self):
        return Memento(self.__text)
    
    def restore_memento(self,memento):
        self.__text = memento.get_text()


class Caretaker:
    def __init__(self) -> None:
        self.__mementos = []

    def add_memento(self,memento):
        self.__mementos.append(memento)

    def get_memento(self,index):
        return self.__mementos[index]
    
editor = TextEditor()

editor.set_text("hello world!")
print(f"текущий текст: {editor.get_text()}")

caretaker = Caretaker()
caretaker.add_memento(editor.creat_memento())

editor.set_text("hello python")
print(f"текущий текст: {editor.get_text()}")

editor.restore_memento(caretaker.get_memento(0))
print(f"текст после востановления: {editor.get_text()}")

текущий текст: hello world!
текущий текст: hello python
текст после востановления: hello world!


In [9]:
from abc import ABC,abstractmethod

class Employee:
    def __init__(self,name,position,departament) -> None:
        self.name = name
        self.position = position
        self.departament = departament

    def __repr__(self):
        return f"{self.name} - {self.position} ({self.departament})"
    
class Departament:
    def __init__(self,name) -> None:
        self.name = name
        self.employees = []

    def add_employee(self,employee):
        self.employees.append(employee)

    def get_iterator(self):
        return EmployeeIterator(self)

class EmployeeIterator:
    def __init__(self,departament) -> None:
        self.departament = departament
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index < len(self.departament.employees):
            employee = self.departament.employees[self.index]
            self.index += 1
            return employee
        else:
            raise StopIteration
        
class Mediator(ABC):
    @abstractmethod
    def send_messege(self,from_employee,to_employee,messege): ...

class DepartamentMediator(Mediator):
    def send_messege(self, from_employee, to_employee, messege):
        print(f"сообщение от {from_employee} для {to_employee}: {messege}")

employee1 = Employee("Alice","Developmer","IT")
employee2 = Employee("Bob","Manager","HR")
employee3 = Employee("Jhon","Analyst","IT")
employee4 = Employee("David","Manager","Finance")

it_departament = Departament("IT")
hr_departament = Departament("HR")
finance_departament = Departament("Finance")

it_departament.add_employee(employee1)
it_departament.add_employee(employee3)
hr_departament.add_employee(employee2)
finance_departament.add_employee(employee4)

it_iterator = it_departament.get_iterator()

for employee in it_iterator:
    print(employee)

mediator = DepartamentMediator()

mediator.send_messege(employee1,employee2,"Hello Bob")
mediator.send_messege(employee2,employee1,"Hello")

Alice - Developmer (IT)
Jhon - Analyst (IT)
сообщение от Alice - Developmer (IT) для Bob - Manager (HR): Hello Bob
сообщение от Bob - Manager (HR) для Alice - Developmer (IT): Hello


In [ ]:
from abc import ABC,abstractmethod

class Observer(ABC):
    @abstractmethod
    def update(self,messege): ...

class ConcreteObserver(Observer):
    def __init__(self,name) -> None:
        self.name = name

    def update(self, messege):
        print(f"Observer {self.name} получил сообщение {messege}")

class Subject:
    def __init__(self) -> None:
        self.__observers = []

    def add_observer(self,observer):
        self.__observers.append(observer)

    def remove_observer(self,observer):
        self.__observers.remove(observer)

    def notify_observers(self,messege):
        for observer in self.__observers:
            observer.update(messege)

    def change_state(self,messege):
        print("изменилось состояние")
        self.notify_observers(messege)

subject = Subject()

observer1 = ConcreteObserver("Kristofer")
observer2 = ConcreteObserver("Oleg")

subject.add_observer(observer1)
subject.add_observer(observer2)

subject.change_state("состояние изменилось")

subject.remove_observer(observer2)

subject.change_state("состояние снова изменилось")


изменилось состояние
Observer Kristofer получил сообщение состояние изменилось
Observer Oleg получил сообщение состояние изменилось
изменилось состояние
Observer Kristofer получил сообщение состояние снова изменилось


In [12]:
from abc import ABC,abstractmethod

class State(ABC):
    @abstractmethod
    def handle(self,context): ...

class StateA(State):
    def handle(self, context):
        print("обработка состояния A")
        context.state = StateB()

class StateB(State):
    def handle(self, context):
        print("обработка состояния B")
        context.state = StateA()

class Context:
    def __init__(self) -> None:
        self.state = StateA()

    def request(self):
        self.state.handle(self)

context = Context()

context.request()
context.request()
context.request()

обработка состояния A
обработка состояния B
обработка состояния A


In [14]:
from abc import ABC,abstractmethod

class Strategy(ABC):
    @abstractmethod
    def execute(self,arr): ...

class BubbleSor(Strategy):
    def execute(self,arr):
        n = len(arr)
        for i in range(n - 1):

            for j in range(n - 1 - i):

                if arr[j] > arr[j + 1]:

                    arr[j], arr[j + 1] = arr[j + 1], arr[j]

        return arr
class QuickSort(Strategy):    
    def execute(self,arr):
        if len(arr) <= 1:
            return arr

        pivot = arr[0]

        left = []
        right = []
        equal = []
        for x in arr:

            if x < pivot:
                left.append(x) 
            elif x > pivot:
                right.append(x)
            else:
                equal.append(x) 

        return self.execute(left) + equal + self.execute(right)
    
class Context:
    def __init__(self,strategy: Strategy) -> None:
        self.__strategy = strategy

    def set_strategy(self,strategy):
        self.__strategy = strategy

    def execute_strategy(self,data):
        return self.__strategy.execute(data)
    
context = Context(BubbleSor())

data = [64,32,16,12,22,11,90]
sorted_data = context.execute_strategy(data)
print(f"отсортированные данные при помощи сортировки пузырьком: {sorted_data}")

context.set_strategy(QuickSort())

sorted_data = context.execute_strategy(data.copy())
print(f"отсортированные данные при помощи быстрой сортировки: {sorted_data}")

отсортированные данные при помощи сортировки пузырьком: [11, 12, 16, 22, 32, 64, 90]
отсортированные данные при помощи быстрой сортировки: [11, 12, 16, 22, 32, 64, 90]


In [16]:
from abc import ABC,abstractmethod

class DataProcessor(ABC):
    def Procces_data(self):
        self.load_data()
        self.process()
        self.save_data()

    @abstractmethod
    def load_data(self): ...

    @abstractmethod
    def process(self): ...

    @abstractmethod
    def save_data(self): ...

class TextDataProcessor(DataProcessor):
    def load_data(self):
        print("загрузка текстовых данных")
    def process(self):
        print("обработка текстовых данных")
    def save_data(self):
        print("сохраненееие текстовых данных")

class NumbDataProcessor(DataProcessor):
    def load_data(self):
        print("загрузка числовых данных")
    def process(self):
        print("обработка числовых данных")
    def save_data(self):
        print("сохраненееие числовых данных")

text_processor = TextDataProcessor()
num_processor = NumbDataProcessor()

text_processor.Procces_data()
print()
num_processor.Procces_data()


загрузка текстовых данных
обработка текстовых данных
сохраненееие текстовых данных

загрузка числовых данных
обработка числовых данных
сохраненееие числовых данных


In [2]:
from abc import ABC, abstractmethod

class Observer(ABC):
    @abstractmethod
    def update(self, state): ...

class Subject(ABC):
    def __init__(self) -> None:
        self.__observers = []
    
    def add_observer(self, observer: Observer):
        self.__observers.append(observer)
    
    def remove_observer(self, observer: Observer):
        self.__observers.remove(observer)
    
    def notify_observers(self):
        for observer in self.__observers:
            observer.update(self)

class State(ABC):
    @abstractmethod
    def handle(self, subject): ...

class OnlineState(State):
    def handle(self,subject):
        print("пользователь онлайн")
        subject.notify_observers()
    
class AwayState(State):
    def handle(self,subject):
        print("пользователь отошел")
        subject.notify_observers()

class DoNotDistubState(State):
    def handle(self,subject):
        print("пользователь не беспокоит")
        print("уведомления не будут отправлены")

class User(Subject):
    def __init__(self) -> None:
        super().__init__()
        self.__state = None

    def set_state(self,state:State):
        self.__state = state
        self.__state.handle(self)

class UserObserver(Observer):
    def __init__(self,name:str) -> None:
        self.name = name

    def update(self,state):
        print(f"{self.name} получил уведомление о смене состояния")

user = User()
user.add_observer(UserObserver("наблюдатель 1"))
user.add_observer(UserObserver("наблюдатель 2"))

user.set_state(OnlineState())
user.set_state(AwayState())
user.set_state(DoNotDistubState())

пользователь онлайн
наблюдатель 1 получил уведомление о смене состояния
наблюдатель 2 получил уведомление о смене состояния
пользователь отошел
наблюдатель 1 получил уведомление о смене состояния
наблюдатель 2 получил уведомление о смене состояния
пользователь не беспокоит
уведомления не будут отправлены


In [ ]:
import threading
import time

def task1():
    print("задача 1 началась")
    time.sleep(2)
    print("задача 1 завершилась")

def task2():
    print("задача 2 началась")
    time.sleep(1)
    print("задача 2 завершилась")

thread1 = threading.Thread(target=task1)
thread2 = threading.Thread(target=task2)

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print("все задачи завершены")

задача 1 началась
задача 2 началась
задача 1 завершилась
задача 2 завершилась
все задачи завершены


In [6]:
import threading
import time

def print_NUMBER(number):
    print(f"номер {number}")

numbers = [1,2,3,4,5,6,7,8,9,10]
threads = []

for number in numbers:
    thread = threading.Thread(target=print_NUMBER,args = (number,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

номер 1
номер 2
номер 3
номер 4
номер 5
номер 6
номер 7
номер 8
номер 9
номер 10


In [12]:
import threading

lock = threading.Lock()

shared_data = [0]

def increment():
    with lock:
        for _ in range(100000):
            shared_data[0] += 1

threads = []
for i in range(5):
    thread = threading.Thread(target=increment)
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

print("итоговое значение: ",shared_data[0])

итоговое значение:  500000


In [17]:
import threading
import time

class TaskRunner:
    completed_tasks = 0
    lock = threading.Lock()

    def __init__(self,task_name) -> None:
        self.task_name = task_name

    def run(self):
        print(f"задача {self.task_name} началась!")
        time.sleep(2)
        print(f"\nзадача {self.task_name} завершилась!")
        with TaskRunner.lock:
            TaskRunner.completed_tasks += 1



task1 = TaskRunner("задача 1")
task2 = TaskRunner("задача 2")

thread1 = threading.Thread(target=task1.run)
thread2 = threading.Thread(target=task2.run)

thread1.start()
thread2.start()

thread1.join()
thread2.join()

TaskRunner.completed_tasks

задача задача 1 началась!
задача задача 2 началась!

задача задача 1 завершилась!

задача задача 2 завершилась!


2

In [18]:
import multiprocessing
import time

def task1():
    print("задача 1 началась")
    time.sleep(2)
    print("задача 1 завершилась")

def task2():
    print("задача 2 началась")
    time.sleep(1)
    print("задача 2 завершилась")

thread1 = multiprocessing.Process(target=task1)
thread2 = multiprocessing.Process(target=task2)

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print("все задачи завершены")

все задачи завершены


In [19]:
import multiprocessing

def send_data(queue):
    queue.put("привет из процесса ")

def receive_data(queue):
    messege = queue.get()
    print(f"получено сообщение {messege}")

queue = multiprocessing.Queue()
process1 = multiprocessing.Process(target=send_data,args=(queue,))
process2 = multiprocessing.Process(target=receive_data,args=(queue,))

process1.start()
process2.start()

process1.join()
process2.join()

In [ ]:
import multiprocessing

def append_to_list(shared_list):
    shared_list.append("привет из процесса")

if __name__ == "__main__":

    with multiprocessing.Manager() as manager:
        shared_list = manager.list()

    processes = []
    for _ in range(3):
        process = multiprocessing.Process(target=append_to_list,args=(shared_list,))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()

    print(f"общий список: {shared_list}")

In [3]:
import threading
import time

semaphore = threading.Semaphore(3)

def access_resource(thread_id):
    print(f"поток {thread_id} пытается получить доступ к ресурсу")
    with semaphore:
        print(f"поток {thread_id} получить доступ к ресурсу")
        time.sleep(2)
    print(f"поток {thread_id} освободил ресурс")

threads = []

for i in range(6):
    thread = threading.Thread(target=access_resource,args=(i,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

поток 0 пытается получить доступ к ресурсу
поток 0 получить доступ к ресурсу
поток 1 пытается получить доступ к ресурсу
поток 1 получить доступ к ресурсу
поток 2 пытается получить доступ к ресурсу
поток 2 получить доступ к ресурсу
поток 3 пытается получить доступ к ресурсу
поток 4 пытается получить доступ к ресурсу
поток 5 пытается получить доступ к ресурсу
поток 0 освободил ресурспоток 2 освободил ресурс
поток 4 получить доступ к ресурсу
поток 3 получить доступ к ресурсу
поток 1 освободил ресурс
поток 5 получить доступ к ресурсу

поток 5 освободил ресурс
поток 4 освободил ресурс
поток 3 освободил ресурс


In [2]:
import threading
import time
import random

stock = {
    "item1":50,
    "item2":30,
    "item3":250,
    "item4":130,
}

stock_lock = threading.Lock()

def add_item(stock,item_name,quantity):
    with stock_lock:
        if item_name in stock:
            stock[item_name] += quantity
        else:
            stock[item_name] = quantity
        print(f"добавлены {quantity} единиц товара {item_name}. новое количество {stock[item_name]}")

def remove_item(stock,item_name,quantity):
    with stock_lock:
        if item_name in stock and stock[item_name] >= quantity:
            stock[item_name] -= quantity
            print(f"удалено {quantity} единиц товара {item_name}. новое количество {stock[item_name]}")
        else:
            print(f"не удалось удалить {quantity} единиц товара {item_name}. недостаточно на складе")

def thread_operation(stock):
    for _ in range(random.randint(20,50)):
        operation = random.choice(["add","remove"])   
        item_name = random.choice(list(stock.keys()))
        quantity = random.randint(1,10)  

        if operation == "add":
            add_item(stock,item_name,quantity)
        else:
            remove_item(stock,item_name,quantity)
        time.sleep(random.uniform(0.1,0.5))

threads = []
for _ in range(random.randint(5,10)):
    thread = threading.Thread(target=thread_operation,args=(stock,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

print("\n итоговое количество товаров на складе: ")
for item,quantity in stock.items():
    print(f"{item}:{quantity}")

удалено 4 единиц товара item1. новое количество 46
удалено 8 единиц товара item1. новое количество 38
удалено 9 единиц товара item4. новое количество 121
добавлены 3 единиц товара item1. новое количество 41
удалено 1 единиц товара item4. новое количество 120
добавлены 10 единиц товара item1. новое количество 51
удалено 10 единиц товара item2. новое количество 20
добавлены 7 единиц товара item2. новое количество 27
добавлены 4 единиц товара item3. новое количество 254
добавлены 3 единиц товара item2. новое количество 30
удалено 6 единиц товара item3. новое количество 248
удалено 9 единиц товара item1. новое количество 42
добавлены 1 единиц товара item2. новое количество 31
добавлены 6 единиц товара item3. новое количество 254
добавлены 4 единиц товара item4. новое количество 124
удалено 2 единиц товара item2. новое количество 29
добавлены 5 единиц товара item1. новое количество 47
добавлены 10 единиц товара item4. новое количество 134
удалено 6 единиц товара item3. новое количество 248


In [ ]:
import multiprocessing
import time

def calculate_factorial(n):
    result = 1
    for i in range(1,n+1):
        print(i)
        print(result)
        result *= i
        print("")
    return result

def factorials_in_pararellel(numbers):
    with multiprocessing.Pool(processes= multiprocessing.cpu_count()) as pool:
        results = pool.map(calculate_factorial,numbers)
    return results

if __name__ == "__main__":
    numbers = list(range(1,10))
    start_time = time.time()
    results_parallel = factorials_in_pararellel(numbers)
    end_time = time.time()
    parallel_time = end_time - start_time

    print(f"время выполнения с многозадачностью: {parallel_time:.4f} секунд")

    start_time = time.time()
    results_seq = [calculate_factorial(n) for n in numbers]
    end_time = time.time()
    seq_time = end_time - start_time

    print(f"время выполнения без многозадачности: {seq_time:.4f} секунд")

    print(f"первые 5 результат с многозадачностью: {results_parallel[:5]}")
    print(f"первые 5 результат без многозадачностью: {results_seq[:5]}")

1
1

2
1

3
2

4
6

5
24



120

In [ ]:
import asyncio


async def fetch_data():
    print("начинает загружать данные.....")
    await asyncio.sleep(2)
    print("данные загружены")
    return "результат"

async def convert_data():
    print("начинает обрабатывать данные....")
    await asyncio.sleep(2)
    print("данные обработанны")

ioloop = asyncio.get_event_loop()

tasks = [
    ioloop.create_task(fetch_data()),
    ioloop.create_task(convert_data())
]
tasks_for_wait = asyncio.wait(tasks)

ioloop.run_until_complete(tasks_for_wait)
ioloop.close()

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
import asyncio
from dataclasses import dataclass

@dataclass

class FakeAPI:
    base_url:str

    async def get_data(self,endpoint):
        print(f"отправляем запрос API по адресу {self.base_url}/{endpoint}")
        await asyncio.sleep(2)
        print(f"ответ от API по адресу {self.base_url}/{endpoint} получен")
        return f"данные с {endpoint}"
    
async def main():
    api = FakeAPI("https://sample.ru/")

    data1 = await api.get_data("user/1")
    data2 = await api.get_data("post/1")

    print(data1)
    print(data2)

asyncio.run(main())

In [10]:
from bs4 import BeautifulSoup
import requests

url = "https://ria.ru/"

page = requests.get(url)

print(page.status_code)

filteredNews = []
allNews = []

soup = BeautifulSoup(page.text,"html.parser")

title_tag = soup.find("a",class_="m-ok")
print(title_tag)

200
<a class="m-ok" data-provider="ok" role="button" tabindex="15" title="Войти с помощью Одноклассники"><svg class="svg-icon"><use xlink:href="#social-odnoklassniki" xmlns:xlink="http://www.w3.org/1999/xlink"></use></svg>
                            Одноклассники
                        </a>


In [ ]:
from flask import Flask

app = Flask(__name__)

def calculate_url(first,second,operation):
    match operation:
        case "-":
            return f"<h3>{first-second}</h3>"
        case "+":
            return f"<h3>{first+second}</h3>"
        case "*":
            return f"<h3>{first*second}</h3>"
        case "/":
            return f"<h3>{first/second}</h3>"
        
    

@app.route('/<int:first>-<int:second>')
def minus(first,second):
    return calculate_url(first,second,"-")

@app.route('/<int:first>+<int:second>')
def plus(first,second):
    return calculate_url(first,second,"+")

@app.route('/<int:first>*<int:second>')
def mult(first,second):
    return calculate_url(first,second,"*")

@app.route('/<int:first>/<int:second>')
def sub(first,second):
    return calculate_url(first,second,"/")

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
